# OPENAI-NOTEBOOK6 SENTIMENT ANALYSIS

From course by Jose Portilla

Rate sentiment for companies on Redit posts

In [1]:
import os
import openai
import praw

In [20]:
#os.environ['REDDIT CLIENT ID'] = ''
#os.environ['REDDIT SECRET'] = ''
#os.environ['OPEN_AI_KEY'] = ''

client_id = os.getenv('REDDIT CLIENT ID')
client_secret = os.getenv('REDDIT SECRET')
openai.api_key = os.getenv('OPEN_AI_KEY')

## Reddit API

In [3]:
reddit = praw.Reddit(client_id = client_id,
                    client_secret = client_secret,
                    user_agent = "sentiment analysis test"
                    )

In [5]:
# Get titles from reddit

for submission in reddit.subreddit("finance").hot(limit = 5):
    print(submission.title)

Moronic Monday - April 11, 2023 - Your Weekly Questions Thread
We are Wall Street Journal reporters covering the collapse of SVB. Ask us anything.
De-dollarization has started, but the odds that China's yuan will take over are 'profoundly unlikely to essentially impossible'
Strip private banks of their power to create money
Dour Earnings Loom Over Wall St. as a Slowing Economy Bites


In [6]:
# Another example

for submission in reddit.subreddit("stocks").hot(limit = 5):
    print(submission.title)

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
r/Stocks Daily Discussion & Options Trading Thursday - Apr 13, 2023
Fed expects banking crisis to cause a recession this year, minutes show
Producer price index declines 0.5% in March
Inflation rises 0.1% in March, less than expected


## Retrieve Comments

In [7]:
subreddit_stocks = reddit.subreddit('stocks')

In [8]:
subreddit_stocks.display_name

'stocks'

In [9]:
subreddit_stocks.title

'Stocks - Investing and trading for all'

In [10]:
subreddit_stocks.accounts_active

4752

In [12]:
# Get the top comments and most upvoted

for post in subreddit_stocks.hot(limit = 5):
    print(post.title)
    submission = reddit.submission(post.id)
    
    counter = 0 # print top 2 comments per title
    for comment in submission.comments:
        if comment.body == '[deleted]':
            pass
        else print(comment.body)
        counter +=1
        if counter == 2:
            break
    

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

## Generate Prompts

In [13]:
# Get titles and comments

def get_titles_and_comments(subreddit = 'stocks', limit = 7, num_comments = 3, skip_first = 2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}
    for counter, post in enumerate(subreddit.hot(limit = limit)):
        if counter < skip_first:
            continue
        
        counter += (1-skip_first)
        title_and_comments[counter] = ""
        submission = reddit.submission(post.id)
        title = post.title
        
        title_and_comments[counter] += 'Title: '+title+"\n\n"
        title_and_comments[counter] += 'Comments: \n\n'
        
        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == '[deleted]':
                title_and_comments[counter] += comment.body+"\n"
                comment_counter +=1
            if comment_counter == num_comments:
                break
                
    return title_and_comments
                

In [14]:
titles_and_comments = get_titles_and_comments()
titles_and_comments

{1: "Title: Fed expects banking crisis to cause a recession this year, minutes show\n\nComments: \n\nbro for the past 6 months I've been hearing every single day that recession is either already here or that it is imminent. Which one is it?\nOk so as Brit I am expecting a full on recession in the U.K\nBelieve it or not... Bullish\n",
 2: 'Title: Producer price index declines 0.5% in March\n\nComments: \n\nwhy are we not mooning? makes no sense\nMeanwhile oil is up 15 percent this past month. Something isn’t adding up here.\nHahah April will have much higher inflation all fronts because energy is going up A LOT\n',
 3: 'Title: Inflation rises 0.1% in March, less than expected\n\nComments: \n\nHi, you\'re on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topic/political; feel free to edit it now and be more specific.\n\n**To everyone commenting:**  Please focus on how this affects the stock market or specific s

In [15]:
# Create prompt

def create_prompt(title_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    
    return task+title_and_comments

In [16]:
# Test
print(titles_and_comments[1])

Title: Fed expects banking crisis to cause a recession this year, minutes show

Comments: 

bro for the past 6 months I've been hearing every single day that recession is either already here or that it is imminent. Which one is it?
Ok so as Brit I am expecting a full on recession in the U.K
Believe it or not... Bullish



In [17]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative or neutral. If no ticker or company is mentioned write 'No company mentioned'

Title: Fed expects banking crisis to cause a recession this year, minutes show

Comments: 

bro for the past 6 months I've been hearing every single day that recession is either already here or that it is imminent. Which one is it?
Ok so as Brit I am expecting a full on recession in the U.K
Believe it or not... Bullish



In [21]:
# Function to get sentiment from OpenAI

for key, title_with_comments in titles_and_comments.items():
    prompt = create_prompt(title_with_comments)
    response = openai.Completion.create(engine = 'text-davinci-003',
                                       prompt = prompt,
                                       max_tokens = 256,
                                       temperature = 0,
                                       top_p = 1.0)
    print(title_with_comments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print('-----------------------------')

Title: Fed expects banking crisis to cause a recession this year, minutes show

Comments: 

bro for the past 6 months I've been hearing every single day that recession is either already here or that it is imminent. Which one is it?
Ok so as Brit I am expecting a full on recession in the U.K
Believe it or not... Bullish

Sentiment Report from OpenAI: 
No company mentioned. Neutral
-----------------------------
Title: Producer price index declines 0.5% in March

Comments: 

why are we not mooning? makes no sense
Meanwhile oil is up 15 percent this past month. Something isn’t adding up here.
Hahah April will have much higher inflation all fronts because energy is going up A LOT

Sentiment Report from OpenAI: 
No company mentioned. Neutral sentiment.
-----------------------------
Title: Inflation rises 0.1% in March, less than expected

Comments: 

Hi, you're on r/Stocks, please make sure your post is related to stocks or the stockmarket or it will most likely get removed as being off-topi